In [1]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Collect Stop words
all_stopwords = stopwords.words('english')
print(all_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

In [4]:
words = []
stopwords_within = []
corpus = []
num_lines = len(dataset)

for i in range(0, num_lines):

    # Get words only using regex in re
    line = dataset["Review"].iloc[i]
    line_words = re.findall("[A-Za-z']+", line)
    
    # Make all lower case
    line_words_lower = [None]*len(line_words)
    for index, word in enumerate(line_words):
        line_words_lower[index] = word.lower()
    
    # print(line_words_lower)

    # Split into an array of strings necessary to remove stop words
    line_stopwords = []
    for index, word in enumerate(line_words):
        if word in all_stopwords:
            line_stopwords.append(word)

    influence_stopwords = ["no", "not", "did", "didn't", "only", "above", "had", "hadn't"]
    all_stopwords_adjusted = [stop_word for stop_word in all_stopwords if i not in influence_stopwords]

    # Add back in words that may influence the sentiment such as 'not'
    # Split into an array of strings necessary to remove stop words
    line_words_no_stop_words = [word for word in line_words if word not in all_stopwords_adjusted]

    # Stem the words and filter out stopwords
    stemmer = PorterStemmer()
    porter_stemmed = [stemmer.stem(word) for word in line_words_no_stop_words]

    # Turn back into a string
    review = " ".join(porter_stemmed) 

    # append the string to the total
    corpus.append(review)

In [5]:
def cleanup(dataset):

    words = []
    stopwords_within = []
    corpus = []
    num_lines = len(dataset)

    for i in range(0, num_lines):

        # Get words only using regex in re
        line = dataset["Review"].iloc[i]
        line_words = re.findall("[A-Za-z']+", line)
        
        # Make all lower case
        line_words_lower = [None]*len(line_words)
        for index, word in enumerate(line_words):
            line_words_lower[index] = word.lower()
        
        # print(line_words_lower)

        # Split into an array of strings necessary to remove stop words
        line_stopwords = []
        for index, word in enumerate(line_words):
            if word in all_stopwords:
                line_stopwords.append(word)

        influence_stopwords = ["no", "not", "did", "didn't", "only", "above", "had", "hadn't"]
        all_stopwords_adjusted = [stop_word for stop_word in all_stopwords if i not in influence_stopwords]

        # Add back in words that may influence the sentiment such as 'not'
        # Split into an array of strings necessary to remove stop words
        line_words_no_stop_words = [word for word in line_words if word not in all_stopwords_adjusted]

        # Stem the words and filter out stopwords
        stemmer = PorterStemmer()
        porter_stemmed = [stemmer.stem(word) for word in line_words_no_stop_words]

        # Turn back into a string
        review = " ".join(porter_stemmed) 

        # append the string to the total
        corpus.append(review)

    return corpus

In [6]:
# Importing the dataset
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

corpus = cleanup(dataset)

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer

# fit to an array using fit_transform
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Set the label (1 for good, 0 for bad)
print(dataset)

                                                Review  Liked
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]


In [10]:
vectorizer.get_feature_names()
x_array = X.toarray()
print(x_array)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
y = dataset["Liked"]
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_array, y, test_size = 0.20, random_state = 0)

# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# # Predicting the Test set results
y_pred = classifier.predict(X_test)

# # Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)

stats =  accuracy_score(y_test, y_pred)
modelLearn = True

In [13]:
print(cm)

[[55 42]
 [11 92]]


In [ ]:
captureString = "this place sucked"

In [25]:
# Build 1 entry dictionary similar to Reviews structure with Review:String    
l = {"Review": ["this place sucked"]}

In [26]:
# Convert into a dataframe
dataOne = pd.DataFrame.from_dict(l)

In [27]:
print(dataOne)

              Review
0  this place sucked


In [28]:
# Cleanup the dataframe
oneline = cleanup(dataOne)

In [29]:
print(oneline)

['place suck']


In [31]:
x = vectorizer.fit_transform(oneline)

XOne_arr = x.toarray()

In [33]:
# Use classifier to predict the value
y_pred = self.classifier.predict(XOne_arr)

[[1 1]]
